In [16]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:256"

In [19]:
from transformers import AutoTokenizer, AutoModel
import torch
torch.cuda.set_device(2)
torch.cuda.empty_cache()
model = AutoModel.from_pretrained("THUDM/chatglm2-6b", trust_remote_code=True).half().cuda()
torch.cuda.empty_cache()
tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm2-6b", trust_remote_code=True)

model = model.eval()

response, history = model.chat(tokenizer, "你有什么要对我说的吗", history=[])
print(response)

Loading checkpoint shards: 100%|██████████| 7/7 [00:11<00:00,  1.62s/it]


如果你有任何问题或需要帮助,请随时告诉我。我很乐意为你提供帮助。


In [7]:
! pip install goto-statement

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3.9 -m pip install --upgrade pip' command.


In [20]:
response, history = model.chat(tokenizer, "请判断如下句子是否是病句，如果是，请判断错误的类别并且返回修改后的结果。句子： 做科学上的事实判断，不同的人从不同的视角、不同的方式为出发点，终究可以获得一致的结论，达成某种程度的一致的意见。", history=[])
print(response)

句子本身没有明显的语法或语义错误,可以被视为健康的句子。


In [ ]:
import csv
import json

jsonfile=open('resp.json','w',encoding='gb2312')
count=1
notwritten=[]
with open('dataset.csv', 'r',encoding='gb2312') as f:
    reader = csv.reader(f)
    next(reader)
    for row in reader:
        text=row[1]
        # print(text)
        with open('prompt.txt','r',encoding='utf-8')as f:
            prompt=f.read()
            # print(prompt)
            prompt=prompt+f"\n#\n{text}\n#"
            # print(prompt)
            response, history = model.chat(tokenizer, prompt, history=[])
            # print(response)
            try:
                respjson=json.loads(response)
                
            except:
                print('not written!')
                notwritten.append(count)
                pass
            json.dump(respjson,jsonfile)
            jsonfile.write(',')
            print(count)
            count+=1
    jsonfile.close()
        

In [23]:
jsonfile.close()

In [4]:
import json
with open('resp.json','r',encoding='utf-8')as f:
    data=json.load(f)
with open('resp.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

In [23]:
with open('resp.csv','w',encoding='utf-8') as f:
    writer=csv.writer(f)
    writer.writerow(data[0].keys())
    for item in data:
        writer.writerow(item.values())

modify&test:

In [24]:
import pandas as pd
import csv
content=[]
with open('resp.csv', 'r') as f:
    reader = csv.reader(f)
    # print(type(reader))
    header = next(reader) # 读取表头
    num_fields = len(header) # 字段数目
    content.append(header)
    for row in reader:
        # print(type(row))
        if len(row) != num_fields:
            print(f"Error: Expected {num_fields} fields in line {reader.line_num}, saw {len(row)}")
            del row[2]
            print('line ',reader.line_num,'has been modified!')
        content.append(row)
with open('resp.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerows(content)


Error: Expected 3 fields in line 38, saw 4
line  38 has been modified!
Error: Expected 3 fields in line 39, saw 4
line  39 has been modified!
Error: Expected 3 fields in line 40, saw 4
line  40 has been modified!
Error: Expected 3 fields in line 41, saw 4
line  41 has been modified!
Error: Expected 3 fields in line 43, saw 4
line  43 has been modified!
Error: Expected 3 fields in line 66, saw 4
line  66 has been modified!
Error: Expected 3 fields in line 67, saw 4
line  67 has been modified!
Error: Expected 3 fields in line 68, saw 4
line  68 has been modified!
Error: Expected 3 fields in line 78, saw 4
line  78 has been modified!
Error: Expected 3 fields in line 105, saw 4
line  105 has been modified!
Error: Expected 3 fields in line 106, saw 4
line  106 has been modified!
Error: Expected 3 fields in line 134, saw 4
line  134 has been modified!
Error: Expected 3 fields in line 135, saw 4
line  135 has been modified!
Error: Expected 3 fields in line 138, saw 4
line  138 has been modifi

transform encoding

In [8]:
import codecs

with open('dataset.csv', 'r', encoding='gb2312') as f:
    reader = csv.reader(f)
    rows = [row for row in reader]

with codecs.open('dataset.csv', 'w', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerows(rows)

In [ ]:
df1=pd.read_csv('dataset.csv',low_memory=False)
print(df1)
# for i in df1:
#     print(i)
# print(df1.keys)
df2=pd.read_csv('resp.csv')
print(df2)
# for i in df2:
    # print(i)
df=pd.merge(df2,df1,on="病句",how='left')
df.to_csv('test.csv', encoding="utf_8_sig")

莫名其妙就能跑了。。。

In [37]:
with open('test.csv','r')as f:
    judge_wrong=0
    modify_wrong=0
    tot_num=0
    reader = csv.reader(f)
    header=next(reader)
    for row in reader:
        if row[2]!=row[5]:
            judge_wrong+=1
        if row[3]!=row[4]:
            modify_wrong+=1
        tot_num+=1
jug_acc=1-judge_wrong/tot_num
mod_acc=1-modify_wrong/tot_num
print("judging accuracy:",jug_acc)
print("modifying cohesion:",mod_acc)
        

judging accuracy: 0.0
modifying cohesion: 0.0
